In [1]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModelForCausalLM

/root/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True).to("cuda:0")
# model.to("cuda:0")

Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 11.97it/s]


In [5]:
# prepare image and text prompt, using the appropriate prompt template
# url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_v1_5_radar.jpg?raw=true"
# image = Image.open(requests.get(url, stream=True).raw)
image1 = Image.open("/root/TFVSN/test_images/test4.jpg")
image2 = Image.open("/root/TFVSN/test_images/test2.jpg")

# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image") 
conversation = [
    {
      "role": "user",
      "content": [
          {"type": "text", "text": "explain the image given to you.they are different"},
          {"type": "image"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
inputs = processor(images=[image1, image2], text=prompt, return_tensors="pt").to("cuda:0")
inputs
# inputs.pixel_values.shape
# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=1000)

print(processor.decode(output[0], skip_special_tokens=True))
# output

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[INST]  
 
explain the image given to you.they are different [/INST] The image you've provided appears to be a colorful, stylized illustration of a bear standing in a forest clearing with a group of smaller animals around it. The bear is depicted in a friendly and welcoming manner, with a smile on its face. The smaller animals seem to be enjoying the bear's company, with some of them perched on the bear's back.

The illustration is rich in detail and color, with the bear and the animals rendered in a way that suggests a warm and inviting atmosphere. The background features a variety of trees and foliage, which adds to the natural setting of the scene.

The image also includes a circular diagram with various labels and numbers around it. This diagram seems to be a chart or graph that is related to the image, possibly indicating some sort of data or performance metrics. The labels include "MMM-Vet," "MMM-Bench," "MMM-TextVQA," "MMM-SQA-IMG," "MMM-SQA-TextVQA," "MMM-SQA-TextVQA," "MMM-SQA